## Step 1 – Load the Cleaned Dataset

In [1]:
import sqlite3, pandas as pd

conn = sqlite3.connect("retail_customer_experience.db")
df = pd.read_sql_query("SELECT * FROM customer_360_cleaned;", conn)
conn.close()

df.head()


DatabaseError: Execution failed on sql 'SELECT * FROM customer_360_cleaned;': no such table: customer_360_cleaned